In [0]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
import sys
sys.path.insert(0, "/content/drive/My Drive/Colab Notebooks/CS596/A1")
print(sys.path)

['/content/drive/My Drive/Colab/deeplearning/A1', '/content/drive/My Drive/Colab/deeplearning/A1', '/content/drive/My Drive/Colab/deeplearning/A1', '/content/drive/My Drive/Colab/deeplearning/A1', '', '/env/python', '/usr/lib/python36.zip', '/usr/lib/python3.6', '/usr/lib/python3.6/lib-dynload', '/usr/local/lib/python3.6/dist-packages', '/usr/lib/python3/dist-packages', '/usr/local/lib/python3.6/dist-packages/IPython/extensions', '/root/.ipython', '/root/.cache/pykeops-1.2-cpython-36/']


In [0]:
import os
os.chdir("/content/drive/My Drive/Colab Notebooks/CS596/A1")
!ls

 cifar10		      kaggle_submission.py     models
'CS 596 Assignment-1.ipynb'   knn_submission.csv       __pycache__
 data_process.py	      LinearR_submission.csv


In [0]:
!pip install pykeops

In [0]:
import random
import numpy as np
from data_process import get_CIFAR10_data
import math
from models import KNN, LinearRegression, SVM, LogisticRegression
from kaggle_submission import output_submission_csv
%matplotlib inline

# Loading CIFAR-10

In the following cells we determine the number of images for each split and load the images.

In [0]:
# You can change these numbers for experimentation
# For submission we will use the default values 
TRAIN_IMAGES = 49000
VAL_IMAGES = 1000
TEST_IMAGES = 5000

In [0]:
data = get_CIFAR10_data(TRAIN_IMAGES, VAL_IMAGES, TEST_IMAGES)
X_train, y_train = data['X_train'], data['y_train']
X_val, y_val = data['X_val'], data['y_val']
X_test, y_test = data['X_test'], data['y_test']

Convert the sets of images from dimensions of **(N, 3, 32, 32) -> (N, 3072)** where N is the number of images so that each **3x32x32** image is represented by a single vector.

In [0]:
X_train = np.reshape(X_train, (X_train.shape[0], -1))
X_val = np.reshape(X_val, (X_val.shape[0], -1))
X_test = np.reshape(X_test, (X_test.shape[0], -1))

### Get Accuracy

This function computes how well your model performs using accuracy as a metric.

In [0]:
def get_acc(pred, y_test):
    return np.sum(y_test==pred)/len(y_test)*100

# K-Nearest Neighbors

The kNN classifier consists of two stages:

- During training, the classifier takes the training data and simply remembers it
- During testing, kNN classifies every test image by comparing to all training images and selecting the class that is most common among the k most similar training examples

In this exercise you will implement these steps using writing efficient, vectorized code. Your final implementation should not use for loops to loop over each of the test and train examples. Instead, you should calculate distances between vectorized forms of the datasets. You may refer to the `scipy.spatial.distance.cdist` function to do this efficiently.

The following code :
- Creates an instance of the KNN classifier class with k = 5
- The train function of the KNN class is trained on the training data
- We use the predict function for predicting testing data labels

### Training KNN

In [0]:
knn = KNN(5)
knn.train(X_train, y_train)

### Find best k on validation

The value of k is an important hyperparameter for the KNN classifier. We will choose the best k by examining the performance of classifiers trained with different k values on the validation set.

It's not necessary to try many different values of k for the purposes of this exercise. You may increase k by a magnitude of 2 each iteration up to around k=100 or something similar to get a sense of classifier performance for different k values.

**Modify the code below to loop though different values of k, train a KNN classifier for each k, and output the validation accuracy for each of the classifiers. Be sure to note your best k below as well.**

In [0]:
# TO DO : Experiment with different values of k
max_acc = 0
best_k = 0
for k in range(5,100,2):
  knn = KNN(k)
  knn.train(X_train, y_train)

  pred_knn = knn.predict(X_val)
  acc = get_acc(pred_knn, y_val)
  if acc > max_acc:
    max_acc = acc
    best_k = k
  print('The validation accuracy is given by : {}, when k={}'.format(acc, k))
  
print('The best validation accuracy is given by : {}, when k={}'.format(max_acc, best_k))

The validation accuracy is given by : 32.6, when k=5
Compiling libKeOpstorch700b1fbcba in /root/.cache/pykeops-1.2-cpython-36//build-libKeOpstorch700b1fbcba:
       formula: ArgKMin_Reduction(Sum(Square((Var(0,3072,0) - Var(1,3072,1)))),7,0)
       aliases: Var(0,3072,0); Var(1,3072,1); 
       dtype  : float32
... Done.
The validation accuracy is given by : 32.300000000000004, when k=7
Compiling libKeOpstorch14525e4de3 in /root/.cache/pykeops-1.2-cpython-36//build-libKeOpstorch14525e4de3:
       formula: ArgKMin_Reduction(Sum(Square((Var(0,3072,0) - Var(1,3072,1)))),9,0)
       aliases: Var(0,3072,0); Var(1,3072,1); 
       dtype  : float32
... Done.
The validation accuracy is given by : 30.8, when k=9
Compiling libKeOpstorch2ad1276b0e in /root/.cache/pykeops-1.2-cpython-36//build-libKeOpstorch2ad1276b0e:
       formula: ArgKMin_Reduction(Sum(Square((Var(0,3072,0) - Var(1,3072,1)))),11,0)
       aliases: Var(0,3072,0); Var(1,3072,1); 
       dtype  : float32
... Done.
The validation a

### Testing KNN

Finally, once you have found the best k according to your experiments on the validation set, retrain a classifier with the best k and test your classifier on the test set.

In [0]:
best_k = 5
knn = KNN(best_k)
knn.train(X_train, y_train)

In [0]:
pred_knn = knn.predict(X_test)
print('The testing accuracy is given by : %f' % (get_acc(pred_knn, y_test)))

Compiling libKeOpstorchd787bbd44a in /root/.cache/pykeops-1.2-cpython-36//build-libKeOpstorchd787bbd44a:
       formula: ArgKMin_Reduction(Sum(Square((Var(0,3072,0) - Var(1,3072,1)))),5,0)
       aliases: Var(0,3072,0); Var(1,3072,1); 
       dtype  : float32
... Done.
The testing accuracy is given by : 33.840000


### KNN Kaggle Submission

Once you are satisfied with your solution and test accuracy output a file to submit your test set predictions to the Kaggle for Assignment 1 KNN. Use the following code to do so:

In [0]:
output_submission_csv('knn_submission.csv', knn.predict(X_test))

# LinearRegression


### Train LinearRegression

In [0]:
LinearR = LinearRegression()
LinearR.train(X_train, y_train)

In [0]:
pred_LinearR = LinearR.predict(X_train)
print('The training accuracy is given by : %f' % (get_acc(pred_LinearR, y_train)))

The training accuracy is given by : 8.622449


### Validation

In [0]:
lrs = [0.01, 0.1, 0.5, 1, 1.5, 2, 2.5]
epochs = [50, 100, 250, 500, 1000, 1600]

In [0]:
MaxL = 0
Mrate = 0
Mepoch = 0
for lr in lrs:
  for epoch in epochs:
    LinearR = LinearRegression(lr=lr, epochs=epoch)
    LinearR.train(X_train, y_train)
    pred_LinearR = LinearR.predict(X_val)
    predL = get_acc(pred_LinearR, y_val)
    if predL > MaxL:
      MaxL = predL
      Mrate = lr
      Mepoch = epoch
    print('The validation accuracy is given by : {} when learning rate = {},number of epochs = {}'.format(predL,lr,epoch))
print('The best validation accuracy is given by : {} when learning rate = {},number of epochs = {}'.format(MaxL,Mrate,Mepoch))

The validation accuracy is given by : 8.799999999999999 when learning rate = 0.01,number of epochs = 50
The validation accuracy is given by : 7.1 when learning rate = 0.01,number of epochs = 100
The validation accuracy is given by : 13.4 when learning rate = 0.01,number of epochs = 250
The validation accuracy is given by : 6.800000000000001 when learning rate = 0.01,number of epochs = 500
The validation accuracy is given by : 11.200000000000001 when learning rate = 0.01,number of epochs = 1000
The validation accuracy is given by : 8.6 when learning rate = 0.01,number of epochs = 1600
The validation accuracy is given by : 12.0 when learning rate = 0.1,number of epochs = 50
The validation accuracy is given by : 7.199999999999999 when learning rate = 0.1,number of epochs = 100
The validation accuracy is given by : 10.5 when learning rate = 0.1,number of epochs = 250
The validation accuracy is given by : 10.7 when learning rate = 0.1,number of epochs = 500
The validation accuracy is given 

### Test Perceptron

In [0]:
LinearRe = LinearRegression(lr=Mrate,epochs=Mepoch)
LinearRe.train(X_train, y_train)

In [0]:
pred_LinearR = LinearRe.predict(X_test)
print('The testing accuracy is given by : %f' % (get_acc(pred_LinearR, y_test)))

The testing accuracy is given by : 11.060000


### LinearRegression Kaggle Submission



In [0]:
output_submission_csv('LinearR_submission.csv', LinearR.predict(X_test))

# Support Vector Machines (with SGD)

Next, you will implement a "soft margin" SVM. In this formulation you will maximize the margin between positive and negative training examples and penalize margin violations using a hinge loss.

We will optimize the SVM loss using SGD. This means you must compute the loss function with respect to model weights. You will use this gradient to update the model weights.

SVM optimized with SGD has 3 hyperparameters that you can experiment with :
- **Learning rate** - similar to as defined above in Perceptron, this parameter scales by how much the weights are changed according to the calculated gradient update. 
- **Epochs** - similar to as defined above in Perceptron.
- **Regularization constant** - Hyperparameter to determine the strength of regularization. In this case it is a coefficient on the term which maximizes the margin.

You will implement the SVM using SGD in the **models/SVM.py**

The following code: 
- Creates an instance of the SVM classifier class 
- The train function of the SVM class is trained on the training data
- We use the predict function to find the training accuracy as well as the testing accuracy

### Train SVM

In [0]:
svm = SVM()
svm.train(X_train, y_train)
pred_svm = svm.predict(X_train)
print('The training accuracy is given by : %f' % (get_acc(pred_svm, y_train)))

The training accuracy is given by : 10.118367


### Validate SVM

In [0]:
SVMlrs = [0.01, 0.1, 1]
SVMepochs = [50, 100, 500]
reg_const = [0.05, 2, 10]
batch_size = [500, 2500]
MaxSVML = 0
MSVMrate = 0
MSVMepoch = 0
MaxSVMreg = 0
MaxSVMbatch = 0
for lr in SVMlrs:
  for epoch in SVMepochs:
    for reg in reg_const:
      for batch in batch_size:
        svm = SVM(lr=lr, epochs=epoch, reg_const=reg, batch_size=batch)
        svm.train(X_train, y_train)
        pred_svm = svm.predict(X_train)
        predsvm = get_acc(pred_svm, y_train)
        if predsvm > MaxSVML:
          MaxSVML = predsvm
          Mrate = lr
          MSVMepoch = epoch
          MaxSVMreg = reg
          MaxSVMbatch = batch
        print('The validation accuracy is given by : {} when learning rate = {},number of epochs = {},regularization constant = {},size = {}'.format(round(predsvm,4),lr,epoch,reg,batch))
print('The best validation accuracy is given by : {} when learning rate = {},number of epochs = {},regularization constant = {},size = {}'.format(round(MaxSVML,4),Mrate,MSVMepoch,MaxSVMreg,MaxSVMbatch))

The validation accuracy is given by : 9.8694 when learning rate = 0.01,number of epochs = 50,regularization constant = 0.05,size = 500
The validation accuracy is given by : 13.3163 when learning rate = 0.01,number of epochs = 50,regularization constant = 0.05,size = 2500
The validation accuracy is given by : 9.0612 when learning rate = 0.01,number of epochs = 50,regularization constant = 2,size = 500
The validation accuracy is given by : 7.4939 when learning rate = 0.01,number of epochs = 50,regularization constant = 2,size = 2500
The validation accuracy is given by : 12.6265 when learning rate = 0.01,number of epochs = 50,regularization constant = 10,size = 500
The validation accuracy is given by : 9.2061 when learning rate = 0.01,number of epochs = 50,regularization constant = 10,size = 2500
The validation accuracy is given by : 8.1571 when learning rate = 0.01,number of epochs = 100,regularization constant = 0.05,size = 500
The validation accuracy is given by : 9.602 when learning r

### Test SVM

In [0]:
SVMRe=SVM(lr=0.01,epochs=50,reg_const=0.05,batch_size=2500)
SVMRe.train(X_train, y_train)
pred_svm = SVMRe.predict(X_test)
print('The testing accuracy is given by : %f' % (get_acc(pred_svm, y_test)))

The testing accuracy is given by : 11.260000


### SVM Kaggle Submission

Once you are satisfied with your solution and test accuracy output a file to submit your test set predictions to the Kaggle for Assignment 1 SVM. Use the following code to do so:

In [0]:
output_submission_csv('svm_submission.csv', svm.predict(X_test))

# LogisticRegression

### Train LogisticRegression

In [0]:
LogisticR = LogisticRegression()
LogisticR.train(X_train, y_train)

In [0]:
pred_LogisticR = LogisticR.predict(X_train)
print('The training accuracy is given by : %f' % (get_acc(pred_LogisticR, y_train)))

The training accuracy is given by : 16.012245


### Validate LogisticRegression

In [0]:
lgrs = [0.01, 0.1, 1, 2, 5]
lgepochs = [50, 100, 250, 500, 1000]

In [0]:
MaxL = 0
Mlgr = 0
Mlgepoch = 0
for lr in lgrs:
  for epoch in lgepochs:
    LogisticR = LogisticRegression(lr=lr, epochs=epoch)
    LogisticR.train(X_train, y_train)
    pred_LogisticR = LogisticR.predict(X_val)
    predL = get_acc(pred_LogisticR, y_val)
    if predL > MaxL:
      MaxL = predL
      Mlgr = lr
      Mlgepoch = epoch
    print('The validation accuracy is given by : {} when learning rate = {},number of epochs = {}'.format(predL,lr,epoch))
print('The best validation accuracy is given by : {} when learning rate = {},number of epochs = {}'.format(MaxL,Mlgr,Mlgepoch))

The validation accuracy is given by : 17.4 when learning rate = 0.01,number of epochs = 50
The validation accuracy is given by : 15.9 when learning rate = 0.01,number of epochs = 100
The validation accuracy is given by : 17.7 when learning rate = 0.01,number of epochs = 250
The validation accuracy is given by : 18.0 when learning rate = 0.01,number of epochs = 500
The validation accuracy is given by : 19.0 when learning rate = 0.01,number of epochs = 1000
The validation accuracy is given by : 16.7 when learning rate = 0.1,number of epochs = 50
The validation accuracy is given by : 16.8 when learning rate = 0.1,number of epochs = 100
The validation accuracy is given by : 18.9 when learning rate = 0.1,number of epochs = 250
The validation accuracy is given by : 16.7 when learning rate = 0.1,number of epochs = 500
The validation accuracy is given by : 17.2 when learning rate = 0.1,number of epochs = 1000
The validation accuracy is given by : 14.899999999999999 when learning rate = 1,numbe

### Testing LogisticRegression

In [0]:
LogisticR = LogisticRegression(lr=0.01,epochs=1000)
LogisticR.train(X_train, y_train)
pred_LogisticR = LogisticR.predict(X_test)
print('The testing accuracy is given by : %f' % (get_acc(pred_LogisticR, y_test)))

The testing accuracy is given by : 20.520000


### LogisticRegression Kaggle Submission



In [0]:
output_submission_csv('LogisticR_submission.csv', LogisticR.predict(X_test))

In [2]:
import tensorflow.compat.v1 as tf
sess = tf.Session(config=tf.ConfigProto(log_device_placement=True))

Device mapping:
/job:localhost/replica:0/task:0/device:XLA_CPU:0 -> device: XLA_CPU device
/job:localhost/replica:0/task:0/device:XLA_GPU:0 -> device: XLA_GPU device
/job:localhost/replica:0/task:0/device:GPU:0 -> device: 0, name: Tesla K80, pci bus id: 0000:00:04.0, compute capability: 3.7



In [3]:
from keras import backend as K
K.tensorflow_backend._get_available_gpus()

Using TensorFlow backend.


['/job:localhost/replica:0/task:0/device:GPU:0']

In [4]:
from __future__ import print_function
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K

batch_size = 128
num_classes = 10
epochs = 12

# input image dimensions
img_rows, img_cols = 28, 28

# the data, split between train and test sets
(x_train, y_train), (x_test, y_test) = mnist.load_data()

if K.image_data_format() == 'channels_first':
    x_train = x_train.reshape(x_train.shape[0], 1, img_rows, img_cols)
    x_test = x_test.reshape(x_test.shape[0], 1, img_rows, img_cols)
    input_shape = (1, img_rows, img_cols)
else:
    x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
    x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
    input_shape = (img_rows, img_cols, 1)

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=input_shape))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))

model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])

model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(x_test, y_test))
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

11493376/11490434 [==============================] - 1s 0us/step
x_train shape: (60000, 28, 28, 1)
60000 train samples
10000 test samples





Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Train on 60000 samples, validate on 10000 samples
Epoch 1/12


60000/60000 [==============================] - 16s 268us/step - loss: 0.2670 - acc: 0.9186 - val_loss: 0.0599 - val_acc: 0.9803
Epoch 2/12
60000/60000 [==============================] - 9s 148us/step - loss: 0.0919 - acc: 0.9733 - val_loss: 0.0423 - val_acc: 0.9850
Epoch 3/12
60000/60000 [==============================] - 9s 148us/step - loss: 0.0694 - acc: 0.9789 - val_loss: 0.0342 - val_acc: 0.9888
Epoch 4/12
60000/60000 [==============================] - 9s 148us/step - loss: 0.0593 - acc: 0.9827 - val_loss: 0.0321 - val_acc: 0.9892
Epoch 5/12
60000/60000 [===============